In [1]:
import sys
import os
from typing import Tuple
import pandas as pd 
import mlflow as mf

## replace with your specific paths
sys.path.append("/home/aamir07/mlops2_with_dagster/") 
ARTIFACT_PATH = "/home/aamir07/mlops2_with_dagster/artifacts/mlflow_artfacts" 
LOGGER_FOLDER_PATH = "/home/aamir07/mlops2_with_dagster/notebooks"

from mlops2_with_dagster import model_pipeline, features_pipeline, encoder_pipeline
from pathlib import Path
from joblib import dump, load
from hamilton import driver, base
from mlops2_with_dagster.utils import get_project_dir, printse
from sklearn.preprocessing import (
    StandardScaler,
    LabelEncoder
)

from ortho import BaseKernel
from ortho.utils import Logger
from ortho.ortho.decorators import task
from ortho.ortho.callbacks import MlFlowCallBack, LoggerCallBack


logger = Logger().logger()

   

        
        

None


2023-09-15 09:03:13.928062: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 09:03:15.057833: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 09:03:15.062576: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-15 09:03:18,181 - DEBUG - Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
2023-09-15 09:03:18.559468: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-15 09:03:18,719 - DEBUG - Creating converter from 7 to 5
2023-09-15 09:03:18,722 - DEBUG - Creating converter from 5 to 7
2

In [2]:
project = 'mlops2_with_dagster'
project_dir = get_project_dir(project)
printse(f'project_dir: {project_dir}')

>>>> project_dir: /home/aamir07/mlops2_with_dagster


In [6]:
encoder_file: Path = project_dir/"intermediate_data/encoder.joblib"
data: Path = project_dir/"data/test.csv"
datatype: str = "dataset"
model_file: Path = project_dir/"warehouse/output.joblib"
infer_type: str = "dataset"

In [7]:
# parameters
df = pd.read_csv(data)
encoders = load(encoder_file)
clfinfo = load(model_file)


/home/aamir07/miniconda3/envs/test_with_poetry/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [10]:
class Infer(BaseKernel):
    
    def __init__(self, callbacks,  experiment_name, run_name, load_from_artifact=False):
        super().__init__(callbacks,
                       experiment_name,
                       run_name,
                       load_from_artifact)
        
        self.index_col = 'passengerid'
        self.target_col = "survived"
        self.cat_cols = ["sex", "cabin", "embarked"]
        self.config = {
            'index_column': self.index_col,
            'target_column': self.target_col,
            'categorical_columns': self.cat_cols
        }
        self.config_model = {
            'index_column': self.index_col,
            'target_column': self.target_col,
            'random_state': 42,
            'max_depth': None,
            'validation_size_fraction': 0.33,
            't': 0.5
            }
            
        self.config_infer2 = {
            'index_column': self.index_col,
            'target_column': self.target_col,
            't': 0.5
            }

    
    @task(build_on_previous_run=True, end_mlflow_run=False)       
    def inference(self, df_test=None, encoders=None, clfinfo=None):
        infer2_adapter = base.SimplePythonGraphAdapter(base.DictResult())
        infer2_dr = driver.Driver(self.config_infer2, 
                                    features_pipeline, model_pipeline, encoder_pipeline,
                                    adapter = infer2_adapter)
        dinfer2 = dict(
            df = df_test,
            clf = clfinfo['fit_clf'],
            **encoders['encoders']
        )
        output_nodes = ['chain_predictions']
        
        output = infer2_dr.execute(output_nodes, inputs  = dinfer2)
        
        data = {"inference_results": output }
        payload = {"artifact_path": ARTIFACT_PATH,
                   "params": self.config_infer2}
        
        return data, payload
    
    def run(self, *args, **kwargs):
        
        return self.inference(*args, **kwargs)
        
        
    

In [11]:
import dagstermill
inferencer = Infer(experiment_name="Mlflow_with_Dagster",
                          run_name="BadamBhum",
                          callbacks = [LoggerCallBack(log_folder=LOGGER_FOLDER_PATH
                                                      , kernel_names=["inference"]),
                                       MlFlowCallBack(kernel_names=["inference"])],
                          load_from_artifact=False)

# mf.end_run() #Un-comment only when a run is ended prematurely or you run into any errors while running the function such that the current run can end!
dagstermill.yield_result(inferencer.run(df_test=df, encoders=encoders, clfinfo=clfinfo)[0]["inference_results"], output_name="inference_results")

2023-09-15 09:05:16,816 - DEBUG - Starting new HTTP connection (1): 127.0.0.1:5002
2023-09-15 09:05:17,887 - DEBUG - http://127.0.0.1:5002 "GET /api/2.0/mlflow/experiments/get-by-name?experiment_name=Mlflow_with_Dagster HTTP/1.1" 200 241
2023-09-15 09:05:17,903 - DEBUG - Resetting dropped connection: 127.0.0.1
2023-09-15 09:05:19,738 - DEBUG - http://127.0.0.1:5002 "GET /api/2.0/mlflow/runs/get?run_uuid=a05f805113ff42a3b54371547a26d49b&run_id=a05f805113ff42a3b54371547a26d49b HTTP/1.1" 200 1572
2023-09-15 09:05:19,748 - DEBUG - Resetting dropped connection: 127.0.0.1
2023-09-15 09:05:20,971 - DEBUG - http://127.0.0.1:5002 "POST /api/2.0/mlflow/runs/update HTTP/1.1" 200 423
2023-09-15 09:05:20,979 - DEBUG - Resetting dropped connection: 127.0.0.1
2023-09-15 09:05:22,176 - DEBUG - http://127.0.0.1:5002 "GET /api/2.0/mlflow/runs/get?run_uuid=a05f805113ff42a3b54371547a26d49b&run_id=a05f805113ff42a3b54371547a26d49b HTTP/1.1" 200 1571
2023-09-15 09:05:22,181 - INFO - Successfuly initiated run

<class 'pandas.core.frame.DataFrame'>


2023-09-15 09:05:22,481 - DEBUG - Computing sex.
2023-09-15 09:05:22,483 - DEBUG - Computing sexencoder.
2023-09-15 09:05:22,485 - DEBUG - Computing sex_category.
2023-09-15 09:05:22,512 - DEBUG - Computing embarked.
2023-09-15 09:05:22,515 - DEBUG - Computing embarkedencoder.
2023-09-15 09:05:22,517 - DEBUG - Computing embarked_category.
2023-09-15 09:05:22,543 - DEBUG - Computing sibsp.
2023-09-15 09:05:22,546 - DEBUG - Computing parch.
2023-09-15 09:05:22,549 - DEBUG - Computing family.
2023-09-15 09:05:22,554 - DEBUG - Computing engineered_features.
2023-09-15 09:05:22,577 - DEBUG - Computing final_imputed_features.
2023-09-15 09:05:22,583 - DEBUG - Computing t.
2023-09-15 09:05:22,587 - DEBUG - Computing chain_predictions.
2023-09-15 09:05:22,657 - DEBUG - Succeed in sending telemetry consisting of [b'{"api_key": "phc_mZg8bkn3yvMxqvZKRlMlxjekFU5DFDdcdAsijJ2EH5e", "event": "os_hamilton_run_start", "properties": {"os_type": "posix", "os_version": "Linux-5.15.90.1-microsoft-standard-

{'chain_predictions': (array([0.82      , 0.15      , 0.2       , ..., 0.20566667, 0.13      ,
         0.26      ]),
  array([1, 0, 0, ..., 0, 0, 0]))}